# Clarifai API

In [ ]:
import os
import json
import time
import pickle

!python -m pip install clarifai-grpc

from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import service_pb2_grpc

stub = service_pb2_grpc.V2Stub(ClarifaiChannel.get_grpc_channel())

from clarifai_grpc.grpc.api import service_pb2, resources_pb2
from clarifai_grpc.grpc.api.status import status_code_pb2

YOUR_CLARIFAI_API_KEY = "your_key"
YOUR_APPLICATION_ID = "my-first-application"
SAMPLE_URL = "https://samples.clarifai.com/metro-north.jpg"

In [ ]:
image_path = "/content/data/images"
images = sorted(os.listdir(image_path))
num_images = len(images)
print("The number of images: {}".format(num_images))

The number of images: 81150


In [ ]:
# This is how you authenticate.
metadata = (("authorization", f"Key {YOUR_CLARIFAI_API_KEY}"),)

img_to_concepts = {}
count = 0
bad_images = []

for img in images:

    start_time = time.time()

    with open(os.path.join(image_path, img), "rb") as f:
        file_bytes = f.read()

    request = service_pb2.PostModelOutputsRequest(
        model_id='general-image-recognition',
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        base64=file_bytes
                    )
                )
            )
        ])

    response = stub.PostModelOutputs(request, metadata=metadata)

    if response.status.code != status_code_pb2.SUCCESS:
        bad_images.append(img)
        continue

    img_to_concepts[img] = response.outputs[0].data.concepts

    count += 1
    time.sleep(0.1)
    end_time = time.time()
    if (count % 100) == 0:
        print("Number of images processed: {}".format(count))
        print("Iteration time: {}".format(end_time - start_time))

In [ ]:
%%time
final_img_to_concepts = {}

for key in img_to_concepts:
    final_img_to_concepts[key] = []
    concepts = list(img_to_concepts[key])
    for c in concepts:
        final_img_to_concepts[key].append(c.name)

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 19.8 ms


In [ ]:
with open("/content/img_concepts.json", "w") as outfile:
    json.dump(concepts, outfile)

with open('bad_images_clarifai.pkl', 'wb') as f:
    pickle.dump(bad_images, f)